In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_path  = 'data/train'
valid_path  = 'data/valid'
test_path  = 'data/test'
train_batches = ImageDataGenerator().flow_from_directory(train_path,
                                                         target_size=(224,224),
                                                         batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224),
                                                         batch_size=30)
test_batches = ImageDataGenerator().flow_from_directory(test_path,
                                                        target_size=(224,224),
                                                        batch_size=50,
                                                        shuffle=False)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [6]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False,
                         input_shape = (224,224, 3))
for layer in base_model.layers:
    layer.trainable = False

In [7]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

x = Flatten()(last_output)

x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy',
                  metrics=['accuracy'])

new_model.fit_generator(train_batches, steps_per_epoch=4,
                        validation_data=valid_batches,
                        validation_steps=2, epochs=20, verbose=2)

C:\Users\darsh\anaconda3\envs\DLVS\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
4/4 - 16s - loss: 0.9768 - accuracy: 0.5500 - val_loss: 3.8714 - val_accuracy: 0.6333
Epoch 2/20
4/4 - 14s - loss: 0.8538 - accuracy: 0.6250 - val_loss: 1.6772 - val_accuracy: 0.6833
Epoch 3/20
4/4 - 13s - loss: 0.6474 - accuracy: 0.6250 - val_loss: 0.8167 - val_accuracy: 0.7667
Epoch 4/20
4/4 - 14s - loss: 0.4324 - accuracy: 0.8750 - val_loss: 0.5630 - val_accuracy: 0.8167
Epoch 5/20
4/4 - 18s - loss: 0.4833 - accuracy: 0.8250 - val_loss: 0.3596 - val_accuracy: 0.8500
Epoch 6/20
4/4 - 16s - loss: 0.4341 - accuracy: 0.8125 - val_loss: 0.4946 - val_accuracy: 0.8833
Epoch 7/20
4/4 - 15s - loss: 0.1148 - accuracy: 0.9688 - val_loss: 0.4147 - val_accuracy: 0.8667
Epoch 8/20
4/4 - 18s - loss: 0.3590 - accuracy: 0.9000 - val_loss: 0.2722 - val_accuracy: 0.9167
Epoch 9/20
4/4 - 15s - loss: 0.1678 - accuracy: 0.9688 - val_loss: 0.4252 - val_accuracy: 0.9000
Epoch 10/20
4/4 - 16s - loss: 0.2560 - accuracy: 0.8500 - val_loss: 0.2234 - val_accuracy: 0.9500
Epoch 11/20
4/4 - 16s - loss:

In [9]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

test_files, test_targets = load_dataset('data/test')
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

100%|██████████| 451/451 [00:01<00:00, 341.41it/s]


15/15 [==============================] - 62s 4s/step - loss: 0.1751 - accuracy: 0.9357